In [1]:
# Import Gurobi

from gurobipy import *
import numpy as np

In [2]:
# Create the model

M = Model(name = 'PI 1');

Academic license - for non-commercial use only


In [3]:
# Team names

names = ['Independiente Medellín', 'Atlético Nacional',
         'Deportivo Cali',         'América de Cali',
         'La Equidad',             'Patriotas',
         'Envigado F.C',           'Rionegro Águilas',
         'Once Caldas',            'Deportivo Pasto',
         'Unión Magdalena',        'Junior',
         'Atlético Huila',         'Deportes Tolima',
         'Jaguares',               'Alianza Petrolera',
         'Atlético Bucaramanga',   'Cúcuta Deportivo',
         'Millonarios',            'Santa Fe']

In [4]:
# Model parameters

# Number of teams
n = 20
T = [i for i in range(n)]

# Number of dates
m = 30
P = [i for i in range(m)]

# Date of non classical matches
c = 15
Q = [i for i in range(m) if i != c]

# Team-pairs that share stadium
S = [(0, 1), (2, 3), (18, 19)]

# Classic matches
C = [(i, i + 1) for i in range(0, n - 1, 2)]

# Prohibited dates for some teams (due to other competitions)
F = {

    # Copa sudamericana
    2 : [21, 29],               # Deportivo Cali
    4 : [17, 25],               # La Equidad
    7 : [17, 25],               # Rionegro Águilas
    8 : [5, 9],                 # Once Caldas

    # Libertadores
    0  : [5, 7],                # Independiente Medellín
    1  : [5, 7, 9, 11],         # Atlético Nacional
    11 : [13, 15, 21, 23, 27],  # Junior
    13 : [13, 15, 21, 23, 27],  # Deportes Tolima

    # Copa Colombia
    3  : [7, 15, 18, 23, 27],   # América de Cali
    5  : [7, 15, 18, 23, 27],   # Patriotas
    6  : [7, 15, 18, 23, 27],   # Envigado
    9  : [7, 15, 18, 23, 27],   # Deportivo Pasto
    10 : [15, 18, 21, 23, 27],  # Unión Magdalena
    12 : [7, 15, 18, 23, 27],   # Atlético Huila
    14 : [7, 15, 18, 23, 27],   # Jaguares
    15 : [7, 15, 18, 23, 27],   # Alianza Petrolera
    16 : [7, 15, 18, 23, 27],   # Atlético Bucaramanga
    17 : [7, 15, 18, 23, 27],   # Cúcuta Deportivo
    18 : [7, 15, 18, 23, 27],   # Millonarios
    19 : [7, 15, 18, 23, 27]    # Santa Fe
}

F2 = F.copy()
for key in F2.keys():
    dates = np.zeros(m)
    for value in F2[key]:
        dates[value - 1] = 1
    F2[key] = dates

# L_jk = 1 if team j plays as visitant in the date k in
# an external tournament (Copa Libertadores, Sudamericana, etc.)
L = {

    # Libertadores
    0  : [5],          # Independiente Medellín
    1  : [5, 9],       # Atlético Nacional
    11 : [15, 21, 23], # Junior
    13 : [15, 23],     # Deportes Tolima

    # Sudamericana
    2 : [29],          # Deportivo Cali
    4 : [17],          # La Equidad
    7 : [25],          # Rionegro
    8 : [5],           # Once Caldas
}

for i in range(n):
    if not i in L.keys():
        L[i] = []

for key in L.keys():
    dates = np.zeros(m)
    for value in L[key]:
        dates[value - 1] = 1
    L[key] = dates


# Best teams (based on 2019 1)
# Millonarios, Cali, Tolima, América, Nacional, Pasto, Junior & Unión Magdalena
b = 8
B = [18, 2, 13, 3, 1, 9, 11, 10]


# Worst teams (based on 2019 1)
# Santa Fe, Rionegro, Huila, Bucaramanga, Equidad, Jaguares, Alianza Petrolera, Envigado
w = 8
W = [19, 7, 12, 16, 4, 14, 15, 6]

# Others
# Medellín, Patriotas, Once Caldas, Cúcuta Deportivo
O = [0, 5, 8, 17] 


# Gains for day of matches and importance of the teams
e = {
     #          week          end
     #       B   W   -     B   W   - 
    'B' : [[.7, .2, .3], [ 1, .4, .5]],
    'W' : [[.3, .2, .4], [.5, .9, .6]],
    'O' : [[.6, .5, .5], [.7, .6, .6]],
}

In [5]:
# Inizializate the variables

# X_ijk = 1 if team i plays against team j in date k
X = []

for i in T:
    row = []
    for j in T:
        row.append([None] * m)
    X.append(row)

for i in T:
    for j in T:
        for k in P:
            vname = '{} vs {} in date {}'.format(names[i], names[j], k + 1)
            X[i][j][k] = M.addVar(vtype = GRB.BINARY, name = vname)


# Decision variable (Y_jk = 1 if team j must play as visitor in date k and k + 1)
Y = [None] * n
for i in range(n):
    row = [None] * (m - 1)
    for k in range(m - 1):
        row[k] = M.addVar(lb = 0, name = 'y_{} {}'.format(i, k), vtype = GRB.BINARY)
    Y[i] = row


# Artificial variable
# Value = 0
phi = M.addVar(lb = 0, vtype = GRB.INTEGER, ub = 0);

# Ik = 1 if in date k there are international matches, 0 otherwise
I = [None] * m
for i in range(m):
    I[i] = M.addVar()

In [6]:
# Restrictions

"""
1. All teams play once against each other
"""
M.addConstrs((quicksum(X[i][j][k] + X[j][i][k] for k in Q) == 1 for i in T for j in T if i != j))

"""
2. Each team must play the same number of matches as local as visitor
"""
M.addConstrs((quicksum(X[i][j][k] - X[j][i][k] for j in T if j != i for k in P) == 0 for i in T));

"""
3. Each team can play at most one match each date
"""
M.addConstrs((quicksum(X[i][j][k] + X[j][i][k] for j in T) + F2[i][k] <= 1 for i in T for k in P));

"""
4. Teams that share stadium cannot play as local the same date
"""
M.addConstrs((quicksum(X[i][h][k] + X[j][h][k] for h in T) <= 1 for i, j in S for k in P));

"""
5. Classic matches must be played only once
"""
M.addConstrs((X[i][j][c] + X[j][i][c] == 1 for i, j in C));

"""
6. Classic matches can not be repeated
"""
M.addConstrs((quicksum(X[i][j][k] for k in Q) + X[i][j][c] == 1 for i, j in C));

"""
7. Teams can not play in the forbidden dates
"""
M.addConstrs((quicksum(X[i][j][k - 1] + X[j][i][k - 1] for k in F[i]) == 0 for i in T for j in T if i != j));

In [7]:
# New restrictions [September 5]

"""
8. If Y_jk is 1, the team j must play as visitant in dates k and k + 1
"""
M.addConstrs((quicksum(X[i][j][k] + X[i][j][k + 1] for i in T) + L[j][k] + L[j][k + 1] <= 1 + Y[j][k] for j in T for k in P[:-1]));

"""
9. No more than phi dates as visitant for any team
"""
M.addConstrs((quicksum(Y[j][k] for k in P[:-1]) <= phi for j in T));

"""
10. No more than 3 dates as visitor for any team
"""
M.addConstrs((quicksum(quicksum(X[i][j][k1] for i in T) + L[j][k1] for k1 in range(k, k + 3)) <= 2 for j in T for k in P[:-2]));

In [8]:
# New restrictions [September 12]

"""
11. Half of the matches must be played before classic date
"""
M.addConstrs((quicksum(X[i][j][k] + X[j][i][k] for k in P[:c+1] for j in T) == n // 2 for i in T));

"""
12. Best teams
"""
M.addConstrs((quicksum(X[j][i][k] for j in B if j != i for k in Q) == b // 2 for i in T if not i in B));

M.addConstrs((quicksum(X[j][i][k] for j in B if j != i for k in Q) <= b // 2 for i in B));

M.addConstrs((quicksum(X[j][i][k] for j in B if j != i for k in Q) >= (b // 2) - 1 for i in B));

"""
13. Worst teams
"""
M.addConstrs((quicksum(X[i][j][k] for j in W if j != i for k in Q) == w // 2 for i in T if not i in W));

M.addConstrs(quicksum(X[i][j][k] for j in W if j != i for k in Q) <= w // 2 for i in W);

M.addConstrs(quicksum(X[i][j][k] for j in W if j != i for k in Q) >= (w // 2) - 1 for i in W);

In [9]:
# New restrictions [September 19]

"""
14. Classic matches can't be played in the first 6 dates of the tournament
"""
M.addConstrs(quicksum(X[i][j][k] + X[j][i][k] for k in P[7:]) == 2 for i, j in C);

"""
15. Classic matches must have at least 4 dates between them
"""
M.addConstrs(quicksum(X[i][j][k] + X[j][i][k] for k in P[c - 4 : c + 5]) == 1 for i, j in C);

"""
16. Half of the matches against best teams must be played before classical week 8
"""
M.addConstrs(quicksum(X[i][j][k] + X[j][i][k] for k in P[:c] for i in B) == b // 2 for j in T if not j in B);

M.addConstrs(quicksum(X[i][j][k] + X[j][i][k] for k in P[:c] for i in B) <= b // 2 for j in B);

M.addConstrs(quicksum(X[i][j][k] + X[j][i][k] for k in P[:c] for i in B) >= (b // 2) - 1 for j in B);

In [10]:
# Set an artificial objective

# obj = phi
# M.setObjective(obj, GRB.MINIMIZE)

In [11]:
# New objective [September 30]

# Best against best in weekday
BwB = quicksum(X[i][j][k] * e['B'][0][0] for i in B for j in B for k in P[0:m:2])
# Best against best in weekend
BnB = quicksum(X[i][j][k] * e['B'][1][0] for i in B for j in B for k in P[1:m:2])
# Best against worst in weekday
BwW = quicksum(X[i][j][k] * e['B'][0][1] for i in B for j in W for k in P[0:m:2])
# Best against worst in weekend
BnW = quicksum(X[i][j][k] * e['B'][1][1] for i in B for j in W for k in P[1:m:2])
# Best against others in weekday
BwO = quicksum(X[i][j][k] * e['B'][0][2] for i in B for j in O for k in P[0:m:2])
# Best against others in weekend
BnO = quicksum(X[i][j][k] * e['B'][1][2] for i in B for j in O for k in P[1:m:2])

# Worst against best in weekday
WwB = quicksum(X[i][j][k] * e['W'][0][0] for i in W for j in B for k in P[0:m:2])
# Worst against best in weekend
WnB = quicksum(X[i][j][k] * e['W'][1][0] for i in W for j in B for k in P[1:m:2])
# Worst against worst in weekday
WwW = quicksum(X[i][j][k] * e['W'][0][1] for i in W for j in W for k in P[0:m:2])
# Worst against worst in weekend
WnW = quicksum(X[i][j][k] * e['W'][1][1] for i in W for j in W for k in P[1:m:2])
# Worst against others in weekday
WwO = quicksum(X[i][j][k] * e['W'][0][2] for i in W for j in O for k in P[0:m:2])
# Worst against others in weekend
WnO = quicksum(X[i][j][k] * e['W'][1][2] for i in W for j in O for k in P[1:m:2])

# Other against best in weekday
OwB = quicksum(X[i][j][k] * e['O'][0][0] for i in O for j in B for k in P[0:m:2])
# Other against best in weekend
OnB = quicksum(X[i][j][k] * e['O'][1][0] for i in O for j in B for k in P[1:m:2])
# Other against worst in weekday
OwW = quicksum(X[i][j][k] * e['O'][0][1] for i in O for j in W for k in P[0:m:2])
# Other against worst in weekend
OnW = quicksum(X[i][j][k] * e['O'][1][1] for i in O for j in W for k in P[1:m:2])
# Other against other in weekday
OwO = quicksum(X[i][j][k] * e['O'][0][2] for i in O for j in O for k in P[0:m:2])
# Other against other in weekend
OnO = quicksum(X[i][j][k] * e['O'][1][2] for i in O for j in O for k in P[1:m:2])

obj = BwB + BnB + BwW + BnW + BwO + BnO + WwB + WnB + WwW + WnW + WwO + WnO + OwB + OnB + OwW + OnW + OwO + OnO
#obj = -phi + (1 / n ** 2)*(BwB + BnB + BwW + BnW + BwO + BnO + WwB + WnB + WwW + WnW + WwO + WnO + OwB + OnB + OwW + OnW + OwO + OnO)
M.setObjective(obj, GRB.MAXIMIZE);

In [ ]:
# New restrictions [October 3]



In [12]:
# Search for a feasible solution

# M.setParam(GRB.Param.TimeLimit, 60);
M.optimize();

Optimize a model with 2774 rows, 12581 columns and 164996 nonzeros
Variable types: 0 continuous, 12581 integer (12580 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 1499 rows and 4561 columns
Presolve time: 0.25s
Presolved: 1275 rows, 8020 columns, 66387 nonzeros
Variable types: 0 continuous, 8020 integer (8020 binary)

Root relaxation: objective 1.222000e+02, 12303 iterations, 2.47 seconds
Total elapsed time = 5.08s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  122.20000    0  609          -  122.20000      -     -   10s
     0     0  122.20000    0  646          -  122.20000      -     -   12s
     0     0  122.20000    0  599          -  122.20000      -     -   12s
     0     0  122.20000    0  433          -  122.20000      -  

In [13]:
# Print dates

print()
for variable in M.getVars():
    if variable.X == 1 and 'y' in variable.getAttr('varName'):
        print(variable.getAttr('varName') + "\t" + str(variable.X))

for variable in M.getVars():
    if variable.X == 1 and 'Junior' in variable.getAttr('varName'):
        print(variable.getAttr('varName'))

print('\nPhi:', phi.X)


Independiente Medellín vs Junior in date 3
Deportivo Cali vs Junior in date 6
Rionegro Águilas vs Junior in date 19
Once Caldas vs Junior in date 25
Deportivo Pasto vs Junior in date 8
Unión Magdalena vs Junior in date 30
Junior vs Atlético Nacional in date 18
Junior vs América de Cali in date 22
Junior vs La Equidad in date 14
Junior vs Patriotas in date 24
Junior vs Envigado F.C in date 26
Junior vs Unión Magdalena in date 16
Junior vs Atlético Huila in date 28
Junior vs Deportes Tolima in date 20
Junior vs Jaguares in date 2
Junior vs Cúcuta Deportivo in date 4
Alianza Petrolera vs Junior in date 10
Atlético Bucaramanga vs Junior in date 1
Millonarios vs Junior in date 12
Santa Fe vs Junior in date 17

Phi: 0.0


In [23]:
# Write dates

for k in P:
    file = open('date{}.txt'.format(k), 'w')
    sum_local_matches = 0
    sum_international_matches = 0
    for i in T:
        if L[i][k] == 1:
            sum_international_matches += 1
            file.write('{} {} {}\n'.format(i + 1, 21, k + 1))
        for j in T:
            if X[i][j][k].X == 1:
                sum_local_matches += 1
                file.write('{} {} {}\n'.format(i + 1, j + 1, k + 1))
    print('Local matches in date {}:'.format(k + 1), sum_local_matches)
    print('External matches in date {}:'.format(k + 1), sum_international_matches, '\n')
    file.close()

Local matches in date 1: 6
External matches in date 1: 0 

Local matches in date 2: 10
External matches in date 2: 0 

Local matches in date 3: 3
External matches in date 3: 0 

Local matches in date 4: 9
External matches in date 4: 0 

Local matches in date 5: 2
External matches in date 5: 3 

Local matches in date 6: 9
External matches in date 6: 0 

Local matches in date 7: 2
External matches in date 7: 0 

Local matches in date 8: 10
External matches in date 8: 0 

Local matches in date 9: 4
External matches in date 9: 1 

Local matches in date 10: 10
External matches in date 10: 0 

Local matches in date 11: 3
External matches in date 11: 0 

Local matches in date 12: 10
External matches in date 12: 0 

Local matches in date 13: 2
External matches in date 13: 0 

Local matches in date 14: 10
External matches in date 14: 0 

Local matches in date 15: 0
External matches in date 15: 2 

Local matches in date 16: 10
External matches in date 16: 0 

Local matches in date 17: 7
External